In [1]:
import json, torch, numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [2]:

MODEL_DIR = "wcberta-prachathai67k-best"  # โฟลเดอร์ที่ save_pretrained ไว้
MAX_LEN   = 512
THRESH    = 0.5                           # เกณฑ์เริ่มต้น (ปรับได้)


In [3]:
# ----- load -----
tok = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR).eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

2025-09-29 23:06:34.507719: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-29 23:06:34.626699: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

2025-09-29 23:06:36.443614: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


cuda


CamembertForSequenceClassification(
  (roberta): CamembertModel(
    (embeddings): CamembertEmbeddings(
      (word_embeddings): Embedding(25005, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): CamembertEncoder(
      (layer): ModuleList(
        (0-11): 12 x CamembertLayer(
          (attention): CamembertAttention(
            (self): CamembertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): CamembertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias

In [4]:
with open(f"{MODEL_DIR}/label_names.json", encoding="utf-8") as f:
    LABELS = json.load(f)
sigmoid = torch.nn.Sigmoid()

def predict_with_probs(texts: list[str], threshold :float = THRESH , top_k:int = None):
    """
    คืน:
      - probs_sorted: รายชื่อคลาส + prob เรียงจากมากไปน้อย
      - chosen: รายการคลาสที่ “ผ่านเกณฑ์” (threshold หรือ top_k)
    """
    enc = tok(texts, return_tensors="pt", truncation=True, max_length=MAX_LEN, padding=True)
    enc = {k: v.to(device) for k, v in enc.items()}
    
    with torch.no_grad():
        logits = model(**enc).logits
        probs = sigmoid(logits).cpu().numpy()  # (B, C)

    results = []
    for i, text in enumerate(texts):
        p = probs[i]                                 # ความน่าจะเป็นต่อคลาส
        order = np.argsort(p)[::-1]                  # เรียงมาก→น้อย
        probs_sorted = [(LABELS[j], float(p[j])) for j in order]

        if top_k and top_k > 0:
            chosen = [LABELS[j] for j in order[:top_k]]
        else:
            chosen = []
            for name , prob in probs_sorted:
                
                if prob >= threshold:
                    chosen.append(name)
                else:
                    break
            

        results.append({
            "text": text,
            "probs_sorted": probs_sorted,  # ดูอันดับทั้งหมดได้
            "chosen": chosen               # คำตอบหลายคลาส
        })
    return results


In [35]:
texts = ["วันนี้อากาศดีมาก ฉันไปเที่ยวทะเลกับเพื่อนๆ มันสนุกสุดๆ เลย","สภาพเศรษฐกิจวันนี้ เป็นปีเผาไม่เหมาะแก่การลงทุน"]
enc = tok(texts , return_tensors="pt", truncation=True, max_length=MAX_LEN, padding=True)

for k, v in enc.items():

    print(k, v.shape)

enc = {k: v.to(device) for k, v in enc.items()}

sigmoid = torch.nn.Sigmoid()
with torch.no_grad():
    logits = model(**enc).logits
    probs = sigmoid(logits).cpu().numpy()  # (B, C)
print(f'probabilities: {probs}')

results = []
for i, text in enumerate(texts):
    p = probs[i]                                 # ความน่าจะเป็นต่อคลาส
    order = np.argsort(p)[::-1]                  # เรียงมาก→น้อย

    print(order)
    
    with open(f"{MODEL_DIR}/label_names.json", encoding="utf-8") as f:
        LABELS = json.load(f)

    probs_sorted = [(LABELS[j], float(p[j])) for j in order]
    print(probs_sorted)

    chosen = []
    for name , prob in probs_sorted:
        
        if prob >= THRESH:
            chosen.append(name)
        else:
            break

    results.append({
            "text": text,
            "probs_sorted": probs_sorted,  # ดูอันดับทั้งหมดได้
            "chosen": chosen               # คำตอบหลายคลาส
        })
    
display(results)


input_ids torch.Size([2, 14])
attention_mask torch.Size([2, 14])
probabilities: [[0.094813   0.03203046 0.14694627 0.05094523 0.06387997 0.5966782
  0.0143029  0.0954277  0.00316024 0.00601025 0.00317499 0.01713263]
 [0.12909263 0.013613   0.09683387 0.0191351  0.05163206 0.01593181
  0.844995   0.00625629 0.06561758 0.00528117 0.00231148 0.00903101]]
[ 5  2  7  0  4  3  1 11  6  9 10  8]
[('environment', 0.5966781973838806), ('quality_of_life', 0.1469462662935257), ('culture', 0.09542769938707352), ('politics', 0.09481299668550491), ('social', 0.06387996673583984), ('international', 0.050945233553647995), ('human_rights', 0.032030459493398666), ('education', 0.017132626846432686), ('economics', 0.014302900992333889), ('national_security', 0.006010252051055431), ('ict', 0.0031749894842505455), ('labor', 0.0031602417584508657)]
[ 6  0  2  8  4  3  5  1 11  7  9 10]
[('economics', 0.8449950218200684), ('politics', 0.12909263372421265), ('quality_of_life', 0.09683386981487274), ('labor', 

[{'text': 'วันนี้อากาศดีมาก ฉันไปเที่ยวทะเลกับเพื่อนๆ มันสนุกสุดๆ เลย',
  'probs_sorted': [('environment', 0.5966781973838806),
   ('quality_of_life', 0.1469462662935257),
   ('culture', 0.09542769938707352),
   ('politics', 0.09481299668550491),
   ('social', 0.06387996673583984),
   ('international', 0.050945233553647995),
   ('human_rights', 0.032030459493398666),
   ('education', 0.017132626846432686),
   ('economics', 0.014302900992333889),
   ('national_security', 0.006010252051055431),
   ('ict', 0.0031749894842505455),
   ('labor', 0.0031602417584508657)],
  'chosen': ['environment']},
 {'text': 'สภาพเศรษฐกิจวันนี้ เป็นปีเผาไม่เหมาะแก่การลงทุน',
  'probs_sorted': [('economics', 0.8449950218200684),
   ('politics', 0.12909263372421265),
   ('quality_of_life', 0.09683386981487274),
   ('labor', 0.06561757624149323),
   ('social', 0.05163206160068512),
   ('international', 0.019135095179080963),
   ('environment', 0.015931811183691025),
   ('human_rights', 0.013613003306090832),
 

In [28]:
texts = ["""
‘คำต่อคำ’ เปิดถ้อยแถลงสีหศักดิ์ กล่าวบนเวที UNGA80 ย้ำบทบาทไทยในเวทีโลก-แจงข้อเท็จจริงพิพาทกัมพูชา
สีหศักดิ์ พวงเกตุแก้ว รัฐมนตรีว่าการกระทรวงการต่างประเทศ ขึ้นกล่าวสุนทรพจน์ในการอภิปรายทั่วไปของการประชุมสมัชชาใหญ่แห่งสหประชาชาติ สมัยสามัญ ครั้งที่ 80 ณ สำนักงานใหญ่สหประชาชาติ นครนิวยอร์ก วานนี้ (27 กันยายน)    ประเด็นสำคัญ   เป็นประชาคมหนึ่งเดียว คำมั่นเดียวกัน อนาคตเดียวกัน
"""]  
out = predict_with_probs(texts, threshold=None , top_k=3)  # หรือกำหนด top_k=2
print(out[0]["probs_sorted"])  # ดู 5 คลาสที่คะแนนสูงสุด
print("chosen:", out[0]["chosen"])
print(out)

[('international', 0.9294572472572327), ('politics', 0.8450518250465393), ('national_security', 0.17128147184848785), ('human_rights', 0.14532507956027985), ('social', 0.0762568861246109), ('economics', 0.04198949411511421), ('culture', 0.023264257237315178), ('education', 0.022871769964694977), ('quality_of_life', 0.020220747217535973), ('labor', 0.00455600768327713), ('ict', 0.003268619067966938), ('environment', 0.0032459881622344255)]
chosen: ['international', 'politics', 'national_security']
[{'text': '\n‘คำต่อคำ’ เปิดถ้อยแถลงสีหศักดิ์ กล่าวบนเวที UNGA80 ย้ำบทบาทไทยในเวทีโลก-แจงข้อเท็จจริงพิพาทกัมพูชา\nสีหศักดิ์ พวงเกตุแก้ว รัฐมนตรีว่าการกระทรวงการต่างประเทศ ขึ้นกล่าวสุนทรพจน์ในการอภิปรายทั่วไปของการประชุมสมัชชาใหญ่แห่งสหประชาชาติ สมัยสามัญ ครั้งที่ 80 ณ สำนักงานใหญ่สหประชาชาติ นครนิวยอร์ก วานนี้ (27 กันยายน)    ประเด็นสำคัญ   เป็นประชาคมหนึ่งเดียว คำมั่นเดียวกัน อนาคตเดียวกัน\n', 'probs_sorted': [('international', 0.9294572472572327), ('politics', 0.8450518250465393), ('national_